## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the __`country_club`__ database.

<br>
1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable,
please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

### Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [0]:
%sql 
drop database if exists country_club cascade;
create database country_club;
show databases;

databaseName
country_club
default


### Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you. 
To get started, first upload the three CSV files to the DBFS as depicted in the following figure

![](https://i.imgur.com/QcCruBr.png)


Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [0]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [0]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
-- bookid: integer (nullable = true)
-- facid: integer (nullable = true)
-- memid: integer (nullable = true)
-- starttime: string (nullable = true)
-- slots: integer (nullable = true)

Facilities Schema
root
-- facid: integer (nullable = true)
-- name: string (nullable = true)
-- membercost: double (nullable = true)
-- guestcost: double (nullable = true)
-- initialoutlay: integer (nullable = true)
-- monthlymaintenance: integer (nullable = true)

Members Schema
root
-- memid: integer (nullable = true)
-- surname: string (nullable = true)
-- firstname: string (nullable = true)
-- address: string (nullable = true)
-- zipcode: integer (nullable = true)
-- telephone: string (nullable = true)
-- recommendedby: integer (nullable = true)
-- joindate: string (nullable = true)

### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [0]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [0]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [0]:
%sql
select * from bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [0]:
%sql
select name, membercost from facilities where membercost <> 0


name,membercost
Tennis Court 1,5.0
Tennis Court 2,5.0
Massage Room 1,9.9
Massage Room 2,9.9
Squash Court,3.5


####  Q2: How many facilities do not charge a fee to members?

In [0]:
%sql
select count(*) from facilities where membercost = 0

count(1)
4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [0]:
%sql
select facid, name, membercost, monthlymaintenance from facilities where membercost <> 0 and (membercost / monthlymaintenance) < .20

facid,name,membercost,monthlymaintenance
0,Tennis Court 1,5.0,200
1,Tennis Court 2,5.0,200
4,Massage Room 1,9.9,3000
5,Massage Room 2,9.9,3000
6,Squash Court,3.5,80


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [0]:
%sql
select * from facilities where name like "%2"


facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [0]:
%sql
select name, monthlymaintenance, if(monthlymaintenance > 100, 'expensive','cheap') from facilities

facid,name,monthlymaintenance,"(IF((monthlymaintenance > 100), expensive, cheap))"
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [0]:
%sql
select A.firstname, A.surname, B.facid from members A join (select * from bookings where bookid in (select max(bookid) bookid from bookings where memid <> 0 group by facid)) B on B.memid = A.memid

firstname,surname,facid
David,Jones,5
Ramnaresh,Sarwin,0
Tim,Boothe,1
John,Hunt,2
Darren,Smith,3
Tim,Rownam,4
John,Hunt,6
Gerald,Butters,7
Henry,Worthington-Smyth,8


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [0]:
%sql
select concat(D.fullname, " used ", F.name) from facilities F join (select distinct concat(A.surname, ", ", A.firstname) fullname, B.facid from members A join (select * from bookings where facid < 2 and memid <> 0) B on B.memid = A.memid) D on F.facid = D.facid order by D.fullname, F.name

"concat(fullname, used , name)"
"Bader, Florence used Tennis Court 1"
"Bader, Florence used Tennis Court 2"
"Baker, Anne used Tennis Court 1"
"Baker, Anne used Tennis Court 2"
"Baker, Timothy used Tennis Court 1"
"Baker, Timothy used Tennis Court 2"
"Boothe, Tim used Tennis Court 1"
"Boothe, Tim used Tennis Court 2"
"Butters, Gerald used Tennis Court 1"
"Butters, Gerald used Tennis Court 2"


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [0]:
%sql
select concat(if(M.memid=0, M.surname, concat(M.surname, ", ", M.firstname)), " used the ", F.name, " and paid $", if(B.memid = 0, F.guestcost*B.slots, F.membercost*B.slots)) result 
from members M 
join bookings B on M.memid = B.memid 
join facilities F on F.facid = B.Facid
where B.starttime like '2012-09-14%' and if(B.memid = 0, F.guestcost*B.slots, F.membercost*B.slots) >30
order by if(B.memid = 0, F.guestcost*B.slots, F.membercost*B.slots) desc

result
GUEST used the Massage Room 2 and paid $320.0
GUEST used the Massage Room 1 and paid $160.0
GUEST used the Massage Room 1 and paid $160.0
GUEST used the Massage Room 1 and paid $160.0
GUEST used the Tennis Court 2 and paid $150.0
GUEST used the Tennis Court 2 and paid $75.0
GUEST used the Tennis Court 1 and paid $75.0
GUEST used the Tennis Court 1 and paid $75.0
GUEST used the Squash Court and paid $70.0
"Farrell, Jemima used the Massage Room 1 and paid $39.6"


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [0]:
%sql
select concat(if(M.memid=0, M.surname, concat(M.surname, ", ", M.firstname)), " used the ", F.name, " and paid $", if(B.memid = 0, F.guestcost*B.slots, F.membercost*B.slots)) result 
from members M 
join (select slots, memid, facid, starttime from bookings where starttime like '2012-09-14%') B on M.memid = B.memid 
join (select facid, name, guestcost, membercost from facilities) F on F.facid = B.facid
where if(B.memid = 0, F.guestcost*B.slots, F.membercost*B.slots) >30
order by if(B.memid = 0, F.guestcost*B.slots, F.membercost*B.slots) desc

result
GUEST used the Massage Room 2 and paid $320.0
GUEST used the Massage Room 1 and paid $160.0
GUEST used the Massage Room 1 and paid $160.0
GUEST used the Massage Room 1 and paid $160.0
GUEST used the Tennis Court 2 and paid $150.0
GUEST used the Tennis Court 2 and paid $75.0
GUEST used the Tennis Court 1 and paid $75.0
GUEST used the Tennis Court 1 and paid $75.0
GUEST used the Squash Court and paid $70.0
"Farrell, Jemima used the Massage Room 1 and paid $39.6"


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [0]:
%sql
select F.name, sum(if(B.memid = 0, F.guestcost*B.slots, F.membercost*B.slots)) revenue 
from members M 
join bookings B on M.memid = B.memid 
join facilities F on F.facid = B.Facid
group by F.name
order by sum(if(B.memid = 0, F.guestcost*B.slots, F.membercost*B.slots)) desc

name,revenue
Massage Room 1,50351.600000000275
Massage Room 2,14454.599999999995
Tennis Court 2,14310.0
Tennis Court 1,13860.0
Squash Court,13468.0
Badminton Court,1906.5
Pool Table,270.0
Snooker Table,240.0
Table Tennis,180.0
